In [1]:
import shutil
import sys
import os
from pathlib import Path
import json
import re
import random

In [2]:
root_path = Path('/home/jianghui/dataset/VideoReorder-MovieNet/')
all_path = Path(root_path, 'all')
train_path = Path(root_path, 'train')
val_path = Path(root_path, 'val')
test_in_domain_path = Path(root_path, 'test_in_domain')
test_out_domain_path = Path(root_path, 'test_out_domain')
human_path = Path(root_path, 'human_behavior/')
human_in_path = Path(root_path, 'human_behavior/','in_domain')
human_out_path = Path(root_path, 'human_behavior/','out_domain')


In [13]:
# add info_reordered.json for all/
for group_path in all_path.iterdir():
    
    group_id = int(group_path.stem)
    with open(Path(group_path, "info.json"), 'r') as f_info:
        info_json = json.load(f_info)
    
    shuffled_info_json = info_json

    random.seed(group_id)
    random.shuffle(shuffled_info_json['img_id'])

    random.seed(group_id)
    random.shuffle(shuffled_info_json['shot_id'])

    random.seed(group_id)
    random.shuffle(shuffled_info_json['scene_id'])

    with open(Path(group_path, "info_shuffled.json"), 'w') as f_info_shuffled:
        json.dump(shuffled_info_json, f_info_shuffled, indent=2)    

In [15]:
# split test_in_domain
for group_dir in all_path.iterdir():
    if int(group_dir.stem) >= 8835:
        shutil.copytree(group_dir, Path(test_out_domain_path, group_dir.stem))

In [28]:
# split test_out_domain, train, val
i = 0
train_list = []
val_list = []
test_in_domain_list = []
while i < 8835:
    src_path = Path(all_path, str(i))
    if i % 5 == 0:
        if i % 15 == 0: 
            des_path = Path(val_path, str(i))
            val_list.append(i)
        else:
            des_path = Path(test_in_domain_path, str(i))
            test_in_domain_list.append(i)
    else:
        des_path = Path(train_path, str(i))
        train_list.append(i)
    # shutil.copytree(src_path, des_path)
    i += 1

clip_split_json = {
    "train" : train_list,
    "val" : val_list,
    "test_in_domain" : test_in_domain_list,
    "test_out_domain" : [i for i in range(8835, 10031)]
}

with open(Path(root_path, 'clip_id.json'), 'w') as f:
    json.dump(clip_split_json, f, indent=2)


In [30]:
# split human out domain set
clip_split_json["human_behavior_out_domain"] = []
tmp = []
for idx, clip_id in enumerate(clip_split_json["test_out_domain"]):
    if (idx % 12 == 0):
        src_path = Path(test_out_domain_path, str(clip_id))
        des_path = Path(human_out_path, str(clip_id))
        # shutil.copytree(src_path, des_path)
        tmp.append(clip_id)
clip_split_json["human_behavior_out_domain"] = tmp

In [31]:
# split human in domain set
clip_split_json["human_behavior_in_domain"] = []
for i in range(100):
    clip_id = clip_split_json["test_in_domain"][round(i * 11.78)]
    src_path = Path(test_in_domain_path, str(clip_id))
    des_path = Path(human_in_path, str(clip_id))
    clip_split_json["human_behavior_in_domain"].append(clip_id)
    shutil.copytree(src_path, des_path)

with open(Path(root_path, 'clip_id.json'), 'w') as f:
    json.dump(clip_split_json, f, indent=2)

In [33]:
human_clip_id = {
    "in_domain" : clip_split_json["human_behavior_in_domain"],
    "out_domain" : clip_split_json["human_behavior_out_domain"]
}

with open(Path(human_path, 'clip_id.json'), 'w') as f:
    json.dump(human_clip_id, f, indent=2)

In [9]:
shot_sum_list = [[0, 0, 0] for i in range(20)]
scene_sum_list = [[0, 0, 0] for i in range(20)]

In [12]:
# statistic 
src_path = test_out_domain_path
idx = 2

for dir in src_path.iterdir():
    # read info
    with open(Path(dir, 'info.json'), 'r') as f:
        info_json = json.load(f)
    
    # read shot and list
    shot_sum = len(set(info_json["shot_id"]))
    scene_sum = len(set(info_json["scene_id"]))
    shot_sum_list[shot_sum][idx] += 1
    scene_sum_list[scene_sum][idx] += 1

for i, j in zip(shot_sum_list, scene_sum_list):
    print(f'{i[0]}/{j[0]} | {i[1]}/{j[1]}  | {i[2]}/{j[2]} |')


0/0 | 0/0  | 0/0 |
0/381 | 0/824  | 0/843 |
0/136 | 0/216  | 0/205 |
0/44 | 0/84  | 0/76 |
19/19 | 52/27  | 38/33 |
75/8 | 137/10  | 146/16 |
112/1 | 219/7  | 221/13 |
90/0 | 182/6  | 211/4 |
82/0 | 134/1  | 172/0 |
59/0 | 113/2  | 112/2 |
38/0 | 85/1  | 82/2 |
37/0 | 68/0  | 74/0 |
28/0 | 56/0  | 38/1 |
21/0 | 48/0  | 33/0 |
17/0 | 30/0  | 28/1 |
9/0 | 26/0  | 20/0 |
1/0 | 13/0  | 12/0 |
0/0 | 11/0  | 8/0 |
1/0 | 4/0  | 1/0 |
0/0 | 0/0  | 0/0 |
